In [1]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
import time

In [2]:
DEBUG = True

## sender's config
user_name = "penglz@xxx"
password = "secret"
smtp_server_addr = "smtp.xxx"
smtp_port = 587


## email content config
quiz_distribution = "##distribution##"

k = 1
quiz_on_week_k = "Quiz on week {}".format(k)
title = "#SI112: Advanced Geometry# Your Quiz Score on Week {}".format(k)

content_template = """
        <html>
          <head></head>
          <body>
            <p>Hi {},<br><br>
               your quiz score on week {}: {}<br>
               The distribution for this quiz: {} <br><br>
               Regards,<br>
               Instructors
            </p>
          </body>
        </html>
        """


def send(server, mail, name, score):
    msg = MIMEText(content_template.format(name, k, score, quiz_distribution), 'html')
    msg['From'] = user_name
    msg['Subject'] = title
    if DEBUG:
        msg['To'] = user_name
    else:
        msg['To'] = mail
    server.send_message(msg)


In [3]:
## data loading
excel_file = "D:\\BaiduNetdiskDownload\\bypy\\sist-course" \
             "\\Advanced-Geometry\\quizzes\\quiz-all.xlsx"
data_frame = pd.read_excel(excel_file)

num_students = len(data_frame['Name'])
# print(data_length)
# print(data_frame['姓名'][0], data_frame['Email'][0], data_frame[quiz_on_week_k][0])

In [4]:
## connect to the server
def connect(user_smtp_server, user_smtp_port, user_account, user_password):
    server = smtplib.SMTP(user_smtp_server, user_smtp_port)
    reply = server.ehlo()
    if reply[0] == 250:
        print('Connected!', flush=True)
    else:
        print('Failed to connect!', flush=True)
        exit(1)
    server.starttls()
    reply = server.login(user_account, user_password)
    if reply[0] == 235:
        print('Logged in!', flush=True)
    else:
        server.quit()
        print('Failed to login!', flush=True)
        exit(1)
    return server


smtp_server = connect(smtp_server_addr, smtp_port, user_name, password)


Connected!


Logged in!


In [ ]:
## send the email to each student
for i in range(num_students):
    name = data_frame['Name'][i]
    email = data_frame['Email'][i]
    score = data_frame[quiz_on_week_k][i]
    print("I am sending to #{}, {}, {}#".format(name, email, score))
    
    try:
        send(server=smtp_server, mail=email, name=name, score=score)
        print("Success!", flush=True)
    except:
        smtp_server.quit()
        print("Failed!", flush=True)
        exit(1)
    
    if i % 5 == 4:
        smtp_server.quit()
        print('Waiting for timeout ... ', end='', flush=True)
        time.sleep(65)
        print('Done.', flush=True)
        smtp_server = connect(smtp_server_addr, smtp_port, user_name, password)

smtp_server.quit()